In [13]:
import tkinter as tk
from tkinter import messagebox
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from anticaptchaofficial.recaptchav2proxyless import recaptchaV2Proxyless
import time
import os

API_KEY = "1097d71f9d7f27b94dbb5a4ea58cfe62"

def run_automation(document_number):
    try:
        # Configura o WebDriver para o Chrome com opções adicionais
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument("--start-minimized")  # Maximiza a janela do navegador
        chrome_options.add_argument("--disable-blink-features=AutomationControlled")
        chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
        chrome_options.add_argument("--kiosk-printing")  # Adiciona a opção de impressão sem janela

        # Configurações para salvar automaticamente como PDF
        save_path = "C:\\certidao"
        if not os.path.exists(save_path):
            os.makedirs(save_path)

        prefs = {
            "printing.print_preview_sticky_settings.appState": '{"recentDestinations":[{"id":"Save as PDF","origin":"local","account":""}],"selectedDestinationId":"Save as PDF","version":2}',
            "savefile.default_directory": save_path,
            "download.default_directory": save_path,
            "download.prompt_for_download": False,  # Evita que a caixa "Salvar como" seja mostrada
            "download.directory_upgrade": True
        }

        chrome_options.add_experimental_option("prefs", prefs)

        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

        # Abra o site
        driver.get("https://web.trf3.jus.br/certidao-regional/CertidaoCivelEleitoralCriminal/SolicitarDadosCertidao")

        # Simula um tempo de leitura da página antes de interagir com os elementos
        time.sleep(2)

        # Seleção do tipo de certidão
        dropdowncertidao = driver.find_element(By.ID, "Tipo")
        dropdowncertidao.click()
        time.sleep(1)
        option_to_selectcetidao = driver.find_element(By.XPATH, '//*[@id="Tipo"]/option[2]')
        option_to_selectcetidao.click()

        # Seleção tipo de documento
        dropdowndocumento = driver.find_element(By.ID, "TipoDeDocumento")
        dropdowndocumento.click()
        time.sleep(1)
        option_to_selectdocumento = driver.find_element(By.XPATH, '//*[@id="TipoDeDocumento"]/option[3]')
        option_to_selectdocumento.click()

        # Preenchimento campo documento
        input_field = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="Documento"]'))
        )
        input_field.clear()
        input_field.send_keys(document_number)

        # Simula um tempo de leitura da página antes de interagir com o próximo elemento
        time.sleep(2)

        # Seleção da abrangência
        dropdownregional = driver.find_element(By.ID, 'TipoDeAbrangencia')
        dropdownregional.click()
        time.sleep(1)
        option_to_selectregional = driver.find_element(By.XPATH, '//*[@id="TipoDeAbrangencia"]/option[2]')
        option_to_selectregional.click()

        # Captura da URL da página atual
        page_url = driver.current_url

        # Definição do sitekey do reCAPTCHA
        SITE_KEY = "6LfCILcpAAAAAE8TevMJXsksyHHXJFENz4B2_7wl"

        # Usar o solver do Anti-Captcha para resolver o reCAPTCHA
        solver = recaptchaV2Proxyless()
        solver.set_verbose(1)
        solver.set_key(API_KEY)
        solver.set_website_url(page_url)
        solver.set_website_key(SITE_KEY)

        # Resolver o reCAPTCHA
        g_response = solver.solve_and_return_solution()
        if g_response != 0:
            print("g-response: " + g_response)
            
            # Tente localizar o campo de resposta do reCAPTCHA e insira o token
            driver.execute_script(f"document.getElementById('g-recaptcha-response').innerHTML = '{g_response}'")

            # Simula um tempo de leitura da página antes de submeter o formulário
            time.sleep(2)

            # Submeter o formulário ou realizar a próxima ação necessária
            submit_button = driver.find_element(By.ID, "submit")
            submit_button.click()

            # Acionar o botão de impressão
            printbutton = driver.find_element(By.ID, 'botaoImprimirCertidao')
            printbutton.click()
            time.sleep(4)

            # Simula o comando para impressão
            driver.execute_script('window.print();')

        else:
            print("task finished with error " + solver.error_code)

        # Simula um tempo para finalizar e visualizar o resultado
        time.sleep(5)

        # Verifica se algum arquivo PDF foi salvo no diretório
        pdf_files = [f for f in os.listdir(save_path) if f.endswith('.pdf')]
        if pdf_files:
            # Supondo que o arquivo PDF mais recente seja o que foi gerado
            latest_pdf = max(pdf_files, key=lambda x: os.path.getctime(os.path.join(save_path, x)))
            new_name = f"{document_number}.pdf"
            os.rename(os.path.join(save_path, latest_pdf), os.path.join(save_path, new_name))
            messagebox.showinfo("Sucesso", f"Certidão salva em: {os.path.join(save_path, new_name)}")
        else:
            messagebox.showerror("Erro", "Falha ao salvar o arquivo PDF.")

        # Fecha o navegador após salvar
        driver.quit()
        
    except Exception as e:
        messagebox.showerror("Erro", f"Erro durante a execução: {str(e)}")

# Configuração da interface gráfica
root = tk.Tk()
root.title("Automação de Certidão")
root.geometry("400x200")
root.configure(bg='#1c1f2e')  # Tema dark-blue

# Label e Entry para o número do documento
label_document = tk.Label(root, text="Número do Documento:", bg='#1c1f2e', fg='white')
label_document.pack(pady=10)

entry_document = tk.Entry(root, width=30)
entry_document.pack(pady=5)

# Botão para executar a automação
btn_run = tk.Button(root, text="Executar", command=lambda: run_automation(entry_document.get()), bg='#1c6ea4', fg='white')
btn_run.pack(pady=20)

root.mainloop()


making request to createTask
created task with id 919003775
making request to getTaskResult
task solved
g-response: 03AFcWeA65RL9F76y2Cp74CRLiBXAPD_CrSTAMuk8MNe8rfpqML96ECdodUdsX13r-Qkz6O6JMPw3D9X8yuXuiEk8PxxuGsD52eZIlrZ1mMdUlEJiU9ru2ni3gDAYl6ccF3J-SJgNHOTJtXt4EsQf1Nlw7kiLBTJEmEUGe_44Hd1JE9oi_oAPZShqYSePPw1K1kAqapZn9b_w9DOMl3Gnl1yoyDdb9tw_GdtKRcPlHNCbUVBnQihq9-tD2Wd0nBZNjC3Ev528swVNMuQI_OEMU3-rF1GOzDvBKyDhdLzjINo0kddd9-gYc8IM-pxFXZrNJ7bLKIFrfsp7nNCk6zdKr1W1QX7NVq_K84zGm21Xk96W4mQS62TNgk-sBqTbbS-ZsM664s46QaMgIVMINXPYAuRHocOrqBwA9fakMaUH-cZVuvbJ0YyRcMko5sl-JK-KOmcyKvSsDdFq3REEqmB3iH5_kfwkIkSizlgAwmLtaE5Rhh7CCbtkqN0bFjBQr-lriOjzuqjry-T5jrs0PKkVjdCmWbqdZodOt2btibz2l8FBKoMBmT4DqNKSoEnvzB1w4F72yigo67XOHyGrEDJC86kwEylGJjV5Q7CV1tnOMsIg1ObLkxxDVGX_cCqy0jDDbnCFW8LP8Pdux_sObDLMw3LUgtL-JrmHxuUiIuoG-m_sK_6dsGHWnbmfGzST0Dw7qCSTJY_YbDJe_3-KobumTjDWZNHWG22ReTVObnzKHofBICjpLDKvpYE5zvfErPMKR7jEPck_04-sCC91SO7-KI4dLKXaMTg3g9xypxTaUw9IrtBDPCgv8N-u3lZ14valmDm4gwXeRnOjqzCUqbrAqZBa5ehEjOecFf7u8d_JWYatQtL_bjN

In [16]:
import tkinter as tk
from tkinter import messagebox
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from anticaptchaofficial.recaptchav2proxyless import recaptchaV2Proxyless
import time
import os

API_KEY = "1097d71f9d7f27b94dbb5a4ea58cfe62"

def run_automation(document_number):
    try:
        # Configura o WebDriver para o Chrome com opções adicionais
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument("--window-position=-10000,-10000")  # Modo off-screen
        chrome_options.add_argument("--disable-blink-features=AutomationControlled")
        chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
        chrome_options.add_argument("--kiosk-printing")  # Adiciona a opção de impressão sem janela

        # Configurações para salvar automaticamente como PDF
        save_path = "C:\\certidao"
        if not os.path.exists(save_path):
            os.makedirs(save_path)

        prefs = {
            "printing.print_preview_sticky_settings.appState": '{"recentDestinations":[{"id":"Save as PDF","origin":"local","account":""}],"selectedDestinationId":"Save as PDF","version":2}',
            "savefile.default_directory": save_path,
            "download.default_directory": save_path,
            "download.prompt_for_download": False,  # Evita que a caixa "Salvar como" seja mostrada
            "download.directory_upgrade": True
        }

        chrome_options.add_experimental_option("prefs", prefs)

        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

        # Abra o site
        driver.get("https://web.trf3.jus.br/certidao-regional/CertidaoCivelEleitoralCriminal/SolicitarDadosCertidao")

        # Simula um tempo de leitura da página antes de interagir com os elementos
        time.sleep(2)

        # Seleção do tipo de certidão
        dropdowncertidao = driver.find_element(By.ID, "Tipo")
        dropdowncertidao.click()
        time.sleep(1)
        option_to_selectcetidao = driver.find_element(By.XPATH, '//*[@id="Tipo"]/option[2]')
        option_to_selectcetidao.click()

        # Seleção tipo de documento
        dropdowndocumento = driver.find_element(By.ID, "TipoDeDocumento")
        dropdowndocumento.click()
        time.sleep(1)
        option_to_selectdocumento = driver.find_element(By.XPATH, '//*[@id="TipoDeDocumento"]/option[3]')
        option_to_selectdocumento.click()

        # Preenchimento campo documento
        input_field = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="Documento"]'))
        )
        input_field.clear()
        input_field.send_keys(document_number)

        # Simula um tempo de leitura da página antes de interagir com o próximo elemento
        time.sleep(2)

        # Seleção da abrangência
        dropdownregional = driver.find_element(By.ID, 'TipoDeAbrangencia')
        dropdownregional.click()
        time.sleep(1)
        option_to_selectregional = driver.find_element(By.XPATH, '//*[@id="TipoDeAbrangencia"]/option[2]')
        option_to_selectregional.click()

        # Captura da URL da página atual
        page_url = driver.current_url

        # Definição do sitekey do reCAPTCHA
        SITE_KEY = "6LfCILcpAAAAAE8TevMJXsksyHHXJFENz4B2_7wl"

        # Usar o solver do Anti-Captcha para resolver o reCAPTCHA
        solver = recaptchaV2Proxyless()
        solver.set_verbose(1)
        solver.set_key(API_KEY)
        solver.set_website_url(page_url)
        solver.set_website_key(SITE_KEY)

        # Resolver o reCAPTCHA
        g_response = solver.solve_and_return_solution()
        if g_response != 0:
            print("g-response: " + g_response)
            
            # Tente localizar o campo de resposta do reCAPTCHA e insira o token
            driver.execute_script(f"document.getElementById('g-recaptcha-response').innerHTML = '{g_response}'")

            # Simula um tempo de leitura da página antes de submeter o formulário
            time.sleep(2)

            # Submeter o formulário ou realizar a próxima ação necessária
            submit_button = driver.find_element(By.ID, "submit")
            submit_button.click()

            # Acionar o botão de impressão
            printbutton = driver.find_element(By.ID, 'botaoImprimirCertidao')
            printbutton.click()
            time.sleep(4)

            # Simula o comando para impressão
            driver.execute_script('window.print();')

        else:
            print("task finished with error " + solver.error_code)

        # Simula um tempo para finalizar e visualizar o resultado
        time.sleep(5)

        # Verifica se algum arquivo PDF foi salvo no diretório
        pdf_files = [f for f in os.listdir(save_path) if f.endswith('.pdf')]
        if pdf_files:
            # Supondo que o arquivo PDF mais recente seja o que foi gerado
            latest_pdf = max(pdf_files, key=lambda x: os.path.getctime(os.path.join(save_path, x)))
            new_name = f"{document_number}.pdf"
            os.rename(os.path.join(save_path, latest_pdf), os.path.join(save_path, new_name))
            messagebox.showinfo("Sucesso", f"Certidão salva em: {os.path.join(save_path, new_name)}")
        else:
            messagebox.showerror("Erro", "Falha ao salvar o arquivo PDF.")

        # Fecha o navegador após salvar
        driver.quit()
        
    except Exception as e:
        messagebox.showerror("Erro", f"Erro durante a execução: {str(e)}")

# Configuração da interface gráfica
root = tk.Tk()
root.title("Automação de Certidão")
root.geometry("400x200")
root.configure(bg='#1c1f2e')  # Tema dark-blue

# Label e Entry para o número do documento
label_document = tk.Label(root, text="Número do Documento:", bg='#1c1f2e', fg='white')
label_document.pack(pady=10)

entry_document = tk.Entry(root, width=30)
entry_document.pack(pady=5)

# Botão para executar a automação
btn_run = tk.Button(root, text="Executar", command=lambda: run_automation(entry_document.get()), bg='#1c6ea4', fg='white')
btn_run.pack(pady=20)

root.mainloop()


making request to createTask
created task with id 920678724
making request to getTaskResult
task solved
g-response: 03AFcWeARZOe1hZBBqjpKmw1S6C6ElIsMAnmpvR_WwlNAFH-PLN-0S5Y4mGcamJABMvSJD9u1NuBJCsotLcJYhQs2_edM0WOIdY2PhSTLqme0oG1870QZ_OGY42miXAZmTFq9PJVavldtDPQNTlu2xXL-N2aXUxzr9-XmKju_-36yRUD-4TGKHgfdkznsgUh-uNDemPiOM7O1AsuHmBCy1O4tliJQpLBtgy-_GqZL4BCEWt3qqH0WHN-NUo4zUVtbCP_-mT9HTTSC3yDnEaFA4qx_4xEyUltL7xj2K0rPTiP6oIqrLywD4e1xDeL1vLfyqcybwKeO5Oyh9HpzW-bKR9gg-SowTZY3DkdOa2RMRZJA9q97UNQhWHSZQwhQswuZWnf_U0AheWqE-hqFNJusTtE7GjTap8K560I8b7Pp4Bz9NNOc7WLdhOwEV8XXhAEvGnC-o-k39-95bsY-Ln2BVBiK8xjMx0a6idl8woh4ftNoVPwJkhdv1zE-6B8GIef-0wTs_2ehoIHg5U8ZasosuVj_7sWybFy_2P8FrwHmMeWi5zePonFPDFxGxIptMtWMoj-mcjtSlI_e6Ct-AtZZF5UgEFHY6ddnbKD
making request to createTask
created task with id 920683350
making request to getTaskResult
task is still processing
making request to getTaskResult
task is still processing
making request to getTaskResult
task solved
g-response: 03AFcWeA5B45QbgCiYpBTuu94qTmAamTCZqqyzCeR